# Analytics for Unstructured Data HW2

# Authors: Neel Sheth, Amanda Nguyen, Ryan Lee, Rianna Patel, Troy Austin, Aarib Mohammed (8:30 - 10:30 Cohort)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
pd.set_option('display.max_rows', 100)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

# Task A. Extract about 5-6k reviews. 

In [ ]:
url = "https://www.beeradvocate.com/beer/top-rated/"
#req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = requests.get(url).text
soup = BeautifulSoup(page, "html.parser")

In [ ]:
# Top 250 Beers Page
product_order = [i.td.text if i.td is not None else None for i in soup.find_all('tr')]
product_name = [i.b.text if i.b is not None else None for i in soup.find_all('tr')]
product_company = [i.find('a').text if i.find('a') is not None else None for i in soup.find_all('span', attrs = {'class':'muted'})]
product_type = [i.text.split('|')[0] if len(i.text.split('|'))>0 else None for i in soup.find_all('span', attrs = {'class':'muted'})]
product_num_rat = [str(i.find_all('td')[2:3])[58:str(i.find_all('td')[2:3]).find('</b')].replace(',','') if i is not None else None for i in soup.find_all('tr')]

product_avg_rat = [str(i.find_all('td')[3:4])[58:str(i.find_all('td')[3:4]).find('</b')] if i is not None else None for i in soup.find_all('tr')]
product_prof = [str(i.a)[9:str(i.a).find('><b')-1] if i is not None else None for i in soup.find_all('tr')]

top_250 = pd.DataFrame({'Rank':product_order,
                       'product_name':product_name,
                       'product_company': product_company,
                       'product_type': product_type,
                       '# of Ratings': product_num_rat,
                       'Avg. Rating': product_avg_rat,
                       'Product Profile': product_prof})

top_250 = top_250[1:].copy().reset_index(drop=True)
top_250

In [ ]:
top_250.to_csv("/Users/RyanLee/Desktop/Fall 2022/Analytics for Unstructured Data/HW2/top_250_beers.csv")

In [ ]:
from time import sleep
from random import randint

reviews = pd.DataFrame()

for p in range(len(top_250)):
    sleep(randint(2,8))
    
    page = top_250.loc[p,'Product Profile']
    url = f'https://www.beeradvocate.com{page}'
    html_page = requests.get(url).text
    soup = BeautifulSoup(html_page, "html.parser")

    # User Review Information
    user_info = [i.find('a', class_ = 'username').text for i in soup.findAll('div', attrs = {'id':'rating_fullview_content_2'})]
    product_order1 = [top_250.loc[p,'Rank']]*len(user_info)
    product_name1 = [top_250.loc[p,'product_name']]*len(user_info)
    user_rating = [i.find('span',class_ = 'muted') for i in soup.findAll('div', attrs = {'id':'rating_fullview_content_2'})]
    product_review = [str(i.find('div'))[str(i.find('div')).find('>')+1:-6].replace('<br>','').replace('</br','').replace('\n','')
              for i in soup.findAll('div', attrs = {'id':'rating_fullview_content_2'})]

    reviews_temp = pd.DataFrame({'product_name': product_name1,
                       'product_review': product_review,
                       'user_rating': user_rating})
    reviews = reviews.append(reviews_temp)


In [ ]:
reviews.to_csv("/Users/RyanLee/Desktop/Fall 2022/Analytics for Unstructured Data/HW2/beer_reviews.csv")

In [3]:
df = pd.read_csv("beer_reviews.csv")
df.dropna(inplace=True)
df

,Unnamed: 0,product_name,product_review,user_rating
4,4,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,"<span class=""muted"">Reviewed by <b><a class=""u..."
9,9,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,"<span class=""muted"">Reviewed by <b><a class=""u..."
12,12,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79Holy. Fucking. Shit. K...","<span class=""muted"">Reviewed by <b><a class=""u..."
13,13,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...","<span class=""muted"">Reviewed by <b><a class=""u..."
14,14,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...","<span class=""muted"">Reviewed by <b><a class=""u..."
...,...,...,...,...
9984,24,Expedition Stout - Bourbon Barrel-Aged,"Pours a thick oily black, vigorous pours doesn...","<span class=""muted"">Reviewed by <b><a class=""u..."
9985,25,Expedition Stout - Bourbon Barrel-Aged,2021-11-2712oz bottle served in a pair of Geor...,"<span class=""muted"">Reviewed by <b><a class=""u..."
9986,26,Expedition Stout - Bourbon Barrel-Aged,2018 Vintage12oz bottle poured into a tulipA- ...,"<span class=""muted"">Reviewed by <b><a class=""u..."
9993,33,Expedition Stout - Bourbon Barrel-Aged,Annual beer grab and generally availed in the ...,"<span class=""muted"">Reviewed by <b><a class=""u..."


# Task B. Assume that a customer, who will be using this recommender system, has specified 3 attributes in a product. E.g., one website describes multiple attributes of beer:

In [4]:
def get_count(x): 
    tokens = word_tokenize(str(x))
    stop_words = set(stopwords.words('english'))
    not_removed = []
    for token in tokens:
        lowered = token.lower() 
        if lowered not in stop_words:
            not_removed.append(lowered)
    counter = Counter(not_removed)
    return counter

counts = pd.DataFrame(list(df['product_review'].map(get_count)))
counts_df = pd.DataFrame(counts.sum().sort_values(ascending = False))

In [5]:
counts_df[:100]

,0
",",13940.0
.,12966.0
>,7794.0
:,1575.0
beer,1538.0
head,1113.0
-,1008.0
's,913.0
’,885.0
chocolate,882.0


#### 3 attributes chosen - Light, fruity, smooth

# Task C. Perform a similarity analysis using cosine similarity (without word embeddings) with the 3 attributes specified by the customer and the reviews. 

In [6]:
# "The similarity script should accept as input a file with the product attributes.."
# The commented out code is to accept a csv file that contains a list of attributes and then make a y vector that will be accepted by our cosine similarity function

# attributes = pd.read_csv()
# y = ""
# for attribute in attributes:
#     attribute = attribute.lower()
#     y+= " "
#     y+= attribute
    
    
y = "light fruity smooth"
def get_cosine_similarity(x):
    corpus = [x,y]
    vector = TfidfVectorizer()
    fit_transform = vector.fit_transform(corpus)
    return cosine_similarity(fit_transform[0], fit_transform)[0][1]

df['similarity_score'] = df['product_review'].map(get_cosine_similarity)
df2 = df[df['similarity_score'] != 0.000000]
df3 = df2.drop(columns=['Unnamed: 0','user_rating'])
df3

,product_name,product_review,similarity_score
12,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79Holy. Fucking. Shit. K...",0.059162
13,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",0.025480
14,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",0.101946
18,Kentucky Brunch Brand Stout,Look - fantastic black and thick with a great ...,0.050244
21,Kentucky Brunch Brand Stout,A: Nightfall in appearance that presents a cla...,0.029936
...,...,...,...
9971,Expedition Stout - Bourbon Barrel-Aged,"12oz bottlePKG 11/17/2021, 13.4% ABVI swear I’...",0.031281
9978,Expedition Stout - Bourbon Barrel-Aged,12oz bottled 11/17/2021 poured into a snifter ...,0.038848
9979,Expedition Stout - Bourbon Barrel-Aged,Look - Just fantastic black and thick with a b...,0.036091
9984,Expedition Stout - Bourbon Barrel-Aged,"Pours a thick oily black, vigorous pours doesn...",0.041454


# Task D. For every review, perform a sentiment analysis (using VADER). 

In [7]:
sentiment = SentimentIntensityAnalyzer()
def get_sentiment(review, **kwargs):
    sentiment_score = sentiment.polarity_scores(review)
    positive_meter = round((sentiment_score['pos'] * 100), 2)
    negative_meter = round((sentiment_score['neg'] * 100), 2)
    return positive_meter if kwargs['k'] == 'positive' else negative_meter

df['positive_sentiment_score'] = df['product_review'].apply(get_sentiment, k='positive')
df['negative_sentiment_score'] = df['product_review'].apply(get_sentiment, k='negative')
# df[df['sim_scores_with_attr']==0].shape[0]
df4 = df.drop(columns=['Unnamed: 0','user_rating'])
df4

,product_name,product_review,similarity_score,positive_sentiment_score,negative_sentiment_score
4,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,0.000000,17.2,0.0
9,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,0.000000,17.2,3.0
12,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79Holy. Fucking. Shit. K...",0.059162,17.2,7.7
13,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",0.025480,20.1,5.9
14,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",0.101946,15.3,3.1
...,...,...,...,...,...
9984,Expedition Stout - Bourbon Barrel-Aged,"Pours a thick oily black, vigorous pours doesn...",0.041454,21.7,0.0
9985,Expedition Stout - Bourbon Barrel-Aged,2021-11-2712oz bottle served in a pair of Geor...,0.000000,12.6,6.0
9986,Expedition Stout - Bourbon Barrel-Aged,2018 Vintage12oz bottle poured into a tulipA- ...,0.000000,12.8,0.0
9993,Expedition Stout - Bourbon Barrel-Aged,Annual beer grab and generally availed in the ...,0.000000,11.0,0.0


# Task E. Create an evaluation score for each beer that uses both similarity and sentiment scores. E.g., total score  = average of (similarity score + sentiment score) or a multiplicative model.  

In [8]:
df5 = df4.groupby('product_name')[['positive_sentiment_score', 'similarity_score']].mean()
df5['scaled_positive_sentiment_score'] = (df5['positive_sentiment_score'] - df5['positive_sentiment_score'].min())/(df5['positive_sentiment_score'].max() - df5['positive_sentiment_score'].min())
df5['eval_score'] = df5['scaled_positive_sentiment_score'] + df5['similarity_score'] 
df5.sort_values(by='eval_score', ascending=False)[:10]

,positive_sentiment_score,similarity_score,scaled_positive_sentiment_score,eval_score
product_name,,,,
Dorothy (Wine Barrel Aged),49.875000,0.004953,1.000000,1.004953
Smooth,35.833333,0.024135,0.687963,0.712098
Space Trace,36.150000,0.013811,0.695000,0.708811
Double Dry Hopped Double Citra Daydream,35.766667,0.000000,0.686481,0.686481
Affogato,35.460000,0.000000,0.679667,0.679667
Appervation,35.357143,0.002093,0.677381,0.679474
Abricot Du Fermier,34.525000,0.003792,0.658889,0.662681
Black Tuesday - Rum Barrel-Aged,34.580000,0.000000,0.660111,0.660111
Double Dry Hopped Melcher Street,32.983333,0.030519,0.624630,0.655149


# Now recommend 3 products to the customer. 

Based on Task E, the 3 products we would recommend to the customer are Dorothy (Wine Barrel Aged), Smooth, and Space Trace.	

# Task F. How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity?

In [16]:
import spacy
import en_core_web_md
nlp = en_core_web_md.load()
spacy.load('en_core_web_md')

In [17]:
spacydf = df4
input_comm = nlp(y)
def spacy_score(comment):
    spacy = nlp(comment).similarity(input_comm)
    return spacy
spacydf['spacy_score'] = spacydf['product_review'].apply(spacy_score)

C:\Users\pokem\AppData\Local\Temp\ipykernel_22308\1255720678.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  spacy = nlp(comment).similarity(input_comm)


In [18]:
spacydf = spacydf.groupby('product_name')[['positive_sentiment_score','spacy_score']].mean()
min_score= spacydf['positive_sentiment_score'] .min()
max_score= spacydf['positive_sentiment_score'].max()
spacydf['positive_sentiment_score_scaled'] = (spacydf['positive_sentiment_score']  - min_score) /(max_score - min_score)
spacydf['final_score'] = spacydf['positive_sentiment_score_scaled'] + spacydf['spacy_score']

In [19]:
display(df5.sort_values(by='eval_score', ascending=False)[:10])
(spacydf.sort_values(by='final_score',ascending=False)[:10])
#We can see below that we get different results for spacy vs similarity

,positive_sentiment_score,similarity_score,scaled_positive_sentiment_score,eval_score
product_name,,,,
Dorothy (Wine Barrel Aged),49.875000,0.004953,1.000000,1.004953
Smooth,35.833333,0.024135,0.687963,0.712098
Space Trace,36.150000,0.013811,0.695000,0.708811
Double Dry Hopped Double Citra Daydream,35.766667,0.000000,0.686481,0.686481
Affogato,35.460000,0.000000,0.679667,0.679667
Appervation,35.357143,0.002093,0.677381,0.679474
Abricot Du Fermier,34.525000,0.003792,0.658889,0.662681
Black Tuesday - Rum Barrel-Aged,34.580000,0.000000,0.660111,0.660111
Double Dry Hopped Melcher Street,32.983333,0.030519,0.624630,0.655149


,positive_sentiment_score,spacy_score,positive_sentiment_score_scaled,final_score
product_name,,,,
Dorothy (Wine Barrel Aged),49.875000,0.443089,1.000000,1.443089
Triple Shot,33.600000,0.648358,0.638333,1.286692
The Peach,33.950000,0.631432,0.646111,1.277543
Affogato,35.460000,0.565348,0.679667,1.245015
Abricot Du Fermier,34.525000,0.556826,0.658889,1.215714
Rare Scooop,30.625000,0.635713,0.572222,1.207935
Double Shot,31.414286,0.603476,0.589762,1.193238
Reaction State,33.200000,0.563183,0.629444,1.192627
Appervation,35.357143,0.507232,0.677381,1.184613


Our recommendations would continue to have Dorothy (Wine Barrel Aged), but Smooth and Space Trace would be dropped from the top 3, and replaced with Triple Shot and The Peach. As we can see from the output, Smooth moves down to tenth place, and Space Trace drops from the top 10 completely. 	

# Task G. How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question. 

In [ ]:
top_df = pd.read_csv("top_250_beers.csv")
top_beers = top_df.groupby('product_name')['Avg. Rating'].mean()
top_beers.sort_values(ascending = False)[:3]

In [ ]:
beer_list = top_beers.sort_values(ascending = False)[:3].index
df6 = df5.reset_index()
df6[df6['product_name'].isin(beer_list)]

Based off solely looking at beer ratings from customers and ignoring similarity and sentiment scores, the top 3 beer recommendations are Kentucky Brunch Brand Stout, Vanilla Bean Assassin, and A Deal With The Devil - Triple Oak Aged. Although not bad ratings, when using the top beer attributes of light, fruity, and smooth, we see that the top beer recommendations differ from the ones accounting for similarity and feature sentiment scores. When looking at the evaluation scores of the top customer rated beers, we see that the they are mid-ranged. This indicates that the customer choices are not bad, but there are other beer recommendations that could better align with customer preferences. When compared to previous parts (Tasks E and F), we see that none of the 5 beers that were recommended throughout those tasks (Dorothy (Wine Barrel Aged), Smooth, Space Trace, Triple Shot, and The Peach) are part of the recommendations for Task G.	

# Task H: Using the top four attributes of beer (from word frequency analysis), calculate the lifts between these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among the remaining 9) using the lift values. Explain your method.

In [6]:
import pandas as pd
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [7]:
counts_df[:100]

,0
",",13940.0
.,12966.0
>,7794.0
:,1575.0
beer,1538.0
head,1113.0
-,1008.0
's,913.0
’,885.0
chocolate,882.0


In [8]:
top_4 = ['chocolate', 'dark', 'bourbon', 'sweet']

In [9]:
chosen_beers_df = df[['product_name','product_review']][:114]


agg_reviews_df = chosen_beers_df.groupby('product_name', as_index=False).agg({'product_name' : 'first', 'product_review' : ' '.join})
agg_reviews_df

,product_name,product_review
0,Abraxas - Barrel-Aged,Good. Not noticeably better than normal abraxa...
1,Blessed,375ml bottle poured into a snifter. Quite deca...
2,Heady Topper,Canned on: ???? (Picked up fresh from the brew...
3,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...
4,King JJJuliusss,Canned on 8/9/22; consumed on 9/9/22Pours an o...
5,King Julius,Not every beer is deserving of the hype it rec...
6,Marshmallow Handjee,Midnight black with a dark mocha head. No real...
7,Mornin' Delight,2022 Variant - bottle split 4 ways - pours a a...
8,Vanilla Bean Assassin,"Bottle, graciously shared by Clark at Peg’s. D..."
9,Zenne Y Frontera,"A: Pours crystal clear, dark golden yellow, al..."


In [10]:


Lift_df = pd.DataFrame(top_4, columns=['Top 4 Attributes'])
Lift_df = Lift_df.set_index('Top 4 Attributes')

Lift_df = Lift_df.reindex(columns = Lift_df.columns.tolist() + ['Abraxas - Barrel-Aged','Blessed', 'Heady Topper', 'Kentucky Brunch Brand Stout', 'King JJJuliusss', 'King Julius', 'Marshmallow Handjee', 'Mornin Delight', 'Vanilla Bean Assassin', 'Zenne Y Frontera'])

contain_chocolate = agg_reviews_df[agg_reviews_df['product_review'].str.contains('chocolate')]


Lift_df.at['chocolate', 'Abraxas - Barrel-Aged'] = True
Lift_df.at['chocolate', 'Blessed'] = True
Lift_df.at['chocolate', 'Heady Topper'] = False
Lift_df.at['chocolate', 'Kentucky Brunch Brand Stout'] = True
Lift_df.at['chocolate', 'King JJJuliusss'] = False
Lift_df.at['chocolate', 'King Julius'] = False
Lift_df.at['chocolate', 'Marshmallow Handjee'] = True
Lift_df.at['chocolate', 'Mornin Delight'] = True
Lift_df.at['chocolate', 'Vanilla Bean Assassin'] = True
Lift_df.at['chocolate', 'Zenne Y Frontera'] = False


contain_dark = agg_reviews_df[agg_reviews_df['product_review'].str.contains('dark')]

Lift_df.at['dark', 'Abraxas - Barrel-Aged'] = True
Lift_df.at['dark', 'Blessed'] = True
Lift_df.at['dark', 'Heady Topper'] = True
Lift_df.at['dark', 'Kentucky Brunch Brand Stout'] = True
Lift_df.at['dark', 'King JJJuliusss'] = True
Lift_df.at['dark', 'King Julius'] = False
Lift_df.at['dark', 'Marshmallow Handjee'] = True
Lift_df.at['dark', 'Mornin Delight'] = True
Lift_df.at['dark', 'Vanilla Bean Assassin'] = False
Lift_df.at['dark', 'Zenne Y Frontera'] = True

contain_bourbon = agg_reviews_df[agg_reviews_df['product_review'].str.contains('bourbon')]

Lift_df.at['bourbon', 'Abraxas - Barrel-Aged'] = True
Lift_df.at['bourbon', 'Blessed'] = True
Lift_df.at['bourbon', 'Heady Topper'] = False
Lift_df.at['bourbon', 'Kentucky Brunch Brand Stout'] = True
Lift_df.at['bourbon', 'King JJJuliusss'] = False
Lift_df.at['bourbon', 'King Julius'] = False
Lift_df.at['bourbon', 'Marshmallow Handjee'] = True
Lift_df.at['bourbon', 'Mornin Delight'] = True
Lift_df.at['bourbon', 'Vanilla Bean Assassin'] = True
Lift_df.at['bourbon', 'Zenne Y Frontera'] = False


contain_sweet = agg_reviews_df[agg_reviews_df['product_review'].str.contains('sweet')]

Lift_df.at['sweet', 'Abraxas - Barrel-Aged'] = True
Lift_df.at['sweet', 'Blessed'] = True
Lift_df.at['sweet', 'Heady Topper'] = True
Lift_df.at['sweet', 'Kentucky Brunch Brand Stout'] = True
Lift_df.at['sweet', 'King JJJuliusss'] = True
Lift_df.at['sweet', 'King Julius'] = True
Lift_df.at['sweet', 'Marshmallow Handjee'] = True
Lift_df.at['sweet', 'Mornin Delight'] = True
Lift_df.at['sweet', 'Vanilla Bean Assassin'] = True
Lift_df.at['sweet', 'Zenne Y Frontera'] = True


Lift_df




,Abraxas - Barrel-Aged,Blessed,Heady Topper,Kentucky Brunch Brand Stout,King JJJuliusss,King Julius,Marshmallow Handjee,Mornin Delight,Vanilla Bean Assassin,Zenne Y Frontera
Top 4 Attributes,,,,,,,,,,
chocolate,True,True,False,True,False,False,True,True,True,False
dark,True,True,True,True,True,False,True,True,False,True
bourbon,True,True,False,True,False,False,True,True,True,False
sweet,True,True,True,True,True,True,True,True,True,True


In [11]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [12]:
find_items = apriori(Lift_df, min_support = 0.3, use_colnames = True)

/Users/riannapatel/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [15]:
lift_analysis = association_rules(find_items, metric ="confidence", min_threshold = 0.4)
lift_analysis = lift_analysis.sort_values(by='lift', ascending =False)
lift_analysis

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3236,"(Mornin Delight, Kentucky Brunch Brand Stout, ...",(Heady Topper),0.5,0.5,0.5,1.0,2.0,0.25,inf
3906,(Zenne Y Frontera),"(Blessed, Heady Topper, Marshmallow Handjee, A...",0.5,0.5,0.5,1.0,2.0,0.25,inf
3928,"(Mornin Delight, Zenne Y Frontera, King JJJuli...","(Blessed, Heady Topper)",0.5,0.5,0.5,1.0,2.0,0.25,inf
3925,"(Heady Topper, King JJJuliusss, Mornin Delight...","(Blessed, Zenne Y Frontera)",0.5,0.5,0.5,1.0,2.0,0.25,inf
3924,"(Heady Topper, Zenne Y Frontera, Mornin Deligh...","(Blessed, King JJJuliusss)",0.5,0.5,0.5,1.0,2.0,0.25,inf
...,...,...,...,...,...,...,...,...,...
1642,"(Blessed, Heady Topper, Kentucky Brunch Brand ...",(Mornin Delight),0.5,1.0,0.5,1.0,1.0,0.00,inf
1641,(Abraxas - Barrel-Aged),"(Blessed, Heady Topper, Kentucky Brunch Brand ...",1.0,0.5,0.5,0.5,1.0,0.00,1.0
1640,(Marshmallow Handjee),"(Blessed, Heady Topper, Kentucky Brunch Brand ...",1.0,0.5,0.5,0.5,1.0,0.00,1.0
3801,"(Heady Topper, King JJJuliusss, Abraxas - Barr...","(Blessed, Mornin Delight)",0.5,1.0,0.5,1.0,1.0,0.00,inf


We chose Zenne Y Frontera to compare to another beer, with Blessed being the one that we found to be most similar due to the high lift value between the 2 beers compared to others.